In [26]:
import logging
import sys

root = logging.getLogger()
root.setLevel(logging.INFO)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)

In [27]:
import sys

sys.path.append('../scripts')
sys.path.append('../')

from scripts import graph_osm_loader
from scripts import pipeline, centroids_graph_builder, clustering
from scripts.path_findings import excraction_pfa, dijkstra_pfa
from scripts import utils

In [28]:
    # 'PARIS': 'R71525',
    # 'Prague': 'R435514',
    # 
    # 'DUBAI': 'R3766483',


In [29]:
GRAPH_ID = 'R71525'  # R13470549 R2555133 R3766483
# примеры id есть в graph_osm_loader.py
g = graph_osm_loader.get_graph(GRAPH_ID)
print(len(g.nodes), len(g.edges))

9512 14818


In [30]:
GRAPH_ID = 'R435514'  # R13470549 R2555133 R3766483
# примеры id есть в graph_osm_loader.py
g = graph_osm_loader.get_graph(GRAPH_ID)
print(len(g.nodes), len(g.edges))

21632 29905


In [69]:
GRAPH_ID = 'R435514'  # R13470549 R2555133 R3766483
# примеры id есть в graph_osm_loader.py
g = graph_osm_loader.get_graph(GRAPH_ID)
print(len(g.nodes), len(g.edges))

21632 29905


In [70]:
points = utils.read_points(GRAPH_ID, g, num=1000)

In [71]:
nodes = g.nodes()

In [72]:
import numpy as np


def dst(da, db):
    return ((da['x'] - db['x']) ** 2 + (da['y'] - db['y']) ** 2) ** 0.5 / 360 * 2 * np.pi * 6371.01 * 1000 * 0.56


def node_dst(u, v):
    da = nodes[u]
    db = nodes[v]
    return dst(da, db)

In [73]:
from pympler.asizeof import asizeof

In [83]:

cms_resolver = clustering.LouvainKMeansCommunityResolver(resolution=1000)

t, cg = centroids_graph_builder.CentroidGraphBuilder().build_with_time(g, cms_resolver)
t, asizeof(cg)/1024/1024, asizeof(g)/1024/1024

find centroids:   0%|          | 0/4878 [00:00<?, ?it/s]

find edges:   0%|          | 0/4878 [00:00<?, ?it/s]

(6.3775482177734375, 11.322639465332031, 22.515457153320312)

In [84]:
len(cg.cms) ** 2 * 8 /1024/1024

181.54055786132812

In [85]:
from scripts.path_findings.arc_flags_builder import ArcFlagsBuilder
from scripts.path_findings.alt_builder import RandomAltBuilder, ClusterAltBuilder

baseline = dijkstra_pfa.Dijkstra(g)
# 
# test_alg = excraction_pfa.ExtractionPfa(
#     g = g,
#     cg=cg,
#     upper=dijkstra_pfa.Dijkstra(cg.g),
#     down=dijkstra_pfa.Dijkstra(g)
# )

# test_alg = ArcFlagsBuilder().build(g, cg.cls2hubs)
# test_alg = dijkstra_pfa.AStar(
#     g = g,
#     h = node_dst
# )
# test_alg = ClusterAltBuilder(0.1).build_ch_pfa(g)

In [86]:
len(cg.cms)

4878

In [87]:
from scripts.path_findings.h_search_builder import MinClusterDistance
from scripts.clustering import LouvainKMeansCommunityResolver

test_alg = MinClusterDistance(workers=10).build_astar(g, cg.cms)

  0%|          | 0/10 [00:00<?, ?it/s]

In [88]:
stat = pipeline.PfaComparator(
    baseline=baseline,
    test_algorithm=test_alg,
    points=points,
    workers=10
).compare()

start 0


find paths:   0%|          | 0/100 [00:00<?, ?it/s]

start 1


find paths:   0%|          | 0/100 [00:00<?, ?it/s]

start 2


...

find paths:   0%|          | 0/100 [00:00<?, ?it/s]

start 8


find paths:   0%|          | 0/100 [00:00<?, ?it/s]

start 9


find paths:   0%|          | 0/100 [00:00<?, ?it/s]

In [89]:
stat


        err_max:    0.0000
        err_min:    0.0000
        err_median: 0.0000
        err_mean:   0.0000
        
        acceleration_max:    321.0009
        acceleration_min:    0.8936
        acceleration_median: 24.8948
        acceleration_mean:   33.0873
        